In [7]:
import numpy as np
from scipy.stats import norm

# --- 常數定義 ---
DAYS_PER_YEAR = 365.25
SECONDS_PER_DAY = 24 * 3600

def calculate_optimal_market_making_params(
    asset: str,
    mid_price: float,
    daily_volatility_pct: float,
    target_order_fill_prob: float,
    order_refresh_time_sec: int,
    stop_loss_risk_prob: float,
    max_holding_time_days: float = 30.0,
    profit_factor: float = 2.0
) -> dict:
    """
    根據 GBM 波動率模型，計算最優的合約造市策略參數。

    參數:
    - asset (str): 交易資產名稱 (如 'XRP')
    - mid_price (float): 當前中價 S
    - daily_volatility_pct (float): 日化波動率 (e.g., 0.05 代表 5%)
    - target_order_fill_prob (float): 期望單次掛單的成交機率 p_order (e.g., 0.2)
    - order_refresh_time_sec (int): 訂單刷新時間，用於計算基礎價差的 Delta t
    - stop_loss_risk_prob (float): 允許在最大持倉時間內觸發止損的機率 (e.g., 0.01)
    - max_holding_time_days (float): 估算止損時所用的最大持倉時間 (天)
    - profit_factor (float): 止盈價差相對於基礎價差的倍數 (e.g., 2.0)

    回傳:
    - dict: 包含建議策略參數百分比的字典
    """
    
    # 1. 數據轉換與年化
    daily_volatility = daily_volatility_pct / 100.0  # 轉換為小數 (e.g., 0.05)
    
    # 年化波動率 σ = σ_daily * sqrt(365.25)
    annual_volatility = daily_volatility * np.sqrt(DAYS_PER_YEAR) 
    
    # 掛單時間年化 Delta t_order
    dt_order = order_refresh_time_sec / (DAYS_PER_YEAR * SECONDS_PER_DAY)
    
    # 止損評估時間年化 Delta t_loss
    dt_loss = max_holding_time_days / DAYS_PER_YEAR 

    # 2. 基礎掛單價差 (d_base) 計算
    # 期望成交機率 p_order 對應的標準常態分佈機率 p/2
    p_half_order = target_order_fill_prob / 2.0
    
    # 計算 Z-score (Phi^-1(p/2))，使用 norm.ppf (Percent Point Function)
    # Z_order 是一個負值，我們取其絕對值
    Z_order = norm.ppf(p_half_order) 
    
    # d_base (%) = [σ * sqrt(Delta t_order) * |Z_order|] * 100
    base_spread_pct = (annual_volatility * np.sqrt(dt_order) * np.abs(Z_order)) * 100
    
    # 3. 止盈價差 (d_profit) 計算
    profit_taking_spread_pct = base_spread_pct * profit_factor
    
    # 4. 止損價差 (d_loss) 計算
    # 由於 stop_loss_risk_prob 越小，Z_loss 越負，|Z_loss| 越大，止損價差 d_loss 越大
    p_half_loss = stop_loss_risk_prob / 2.0
    Z_loss = norm.ppf(p_half_loss)
    
    # d_loss (%) = [σ * sqrt(Delta t_loss) * |Z_loss|] * 100
    stop_loss_spread_pct = (annual_volatility * np.sqrt(dt_loss) * np.abs(Z_loss)) * 100

    # 5. 整理輸出結果
    results = {
        "asset": asset,
        "current_mid_price": mid_price,
        
        # 基礎參數
        "order_refresh_time_sec": order_refresh_time_sec,
        
        # 最優價差參數 (對應 Hummingbot 配置)
        "bid_spread": round(base_spread_pct, 4),
        "ask_spread": round(base_spread_pct, 4),
        "long_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "short_profit_taking_spread": round(profit_taking_spread_pct, 4),
        "stop_loss_spread": round(stop_loss_spread_pct, 4),
        
        # 內部計算值 (供參考)
        "Z_score_order": round(np.abs(Z_order), 4),
        "Z_score_stop_loss": round(np.abs(Z_loss), 4)
    }
    
    return results

# ---
# 具體數值範例：XRP 造市策略
# ---

# 假設的 XRP 數據和策略目標
XRP_PARAMS = {
    "asset": "XRP-USDT",
    "mid_price": 2.5,  # 假設 XRP/USDT 當前價格為 0.50
    # 假設日化波動率為 4% (0.04)
    "daily_volatility_pct": 4.0, 
    # 期望每次掛單在 order_refresh_time 內有 25% 的機率成交 (p_order = 0.25)
    "target_order_fill_prob": 0.25, 
    # 訂單每 60 秒刷新一次 (Delta t)
    "order_refresh_time_sec": 60, 
    # 允許在最大持倉時間 (30天) 內，只有 1% 的機率觸發止損 (p_stop_loss = 0.01)
    "stop_loss_risk_prob": 0.01,
    "max_holding_time_days": 30.0,
    "profit_factor": 2.5 # 止盈設為基礎價差的 2.5 倍
}

# 運行計算器
optimal_params = calculate_optimal_market_making_params(**XRP_PARAMS)

print("--- 🔬 XRP 最優造市參數計算結果 ---")
for key, value in optimal_params.items():
    if "_spread" in key or "time_sec" in key:
        unit = "%" if "spread" in key else "秒"
        print(f"{key:<30}: {value:>.4f} {unit}")
    elif key == "current_mid_price":
        print(f"{key:<30}: {value:>.4f} USDT")
    elif key == "asset":
        print(f"{key:<30}: {value}")

--- 🔬 XRP 最優造市參數計算結果 ---
asset                         : XRP-USDT
current_mid_price             : 2.5000 USDT
order_refresh_time_sec        : 60.0000 秒
bid_spread                    : 0.1213 %
ask_spread                    : 0.1213 %
long_profit_taking_spread     : 0.3031 %
short_profit_taking_spread    : 0.3031 %
stop_loss_spread              : 56.4336 %
